In [1]:
from funcoes_auxiliares import *

from sklearn.linear_model import LinearRegression

pd.options.display.float_format = "{:.2f}".format
pd.options.display.max_rows = 6

In [2]:
lista_estados = ["AC", "AL", "AM", "AP", "BA", "CE", "DF", "ES", "GO", "MA", "MT", "MS", "MG", "PA", "PB", "PR", "PE", "PI", "RJ", "RN", "RO", "RS", "RR", "SC", "SE", "SP", "TO"]

In [3]:
df_estados_brasil = None

In [4]:
for i, estado in enumerate(lista_estados):
    nome_arquivo_despesas = "../dados-limpos/aquisicao_por_classe_de_rendimento_e_estado/%s/dados-limpos/%s_alimentacao.csv" % (estado, estado)
    df_despesas = carregar_df_despesas_gerais(nome_arquivo_despesas)
    df_invertido = df_despesas_gerais_invertido(df_despesas, estado, False)
    if(i == 0):
        df_estados_brasil = df_invertido
    else:
        df_estados_brasil = pd.concat([df_estados_brasil, df_invertido])

In [5]:
df_estados_brasil.reset_index(drop=True, inplace=True)

In [6]:
df_estados_brasil

,Arroz,Feijão,Orgânicos,Outros,"Farinhas, féculas e massas",Macarrão,Farinha de trigo,Farinha de mandioca,Outras,Tubérculos e raízes,...,Sanduíches e salgados,Refrigerantes e outras bebidas não alcoólicas,Lanches,"Cervejas, chopes e outras bebidas alcoólicas",Alimentação light e diet,Outras,Tamanho médio da família,Total gasto por classes,Estado,Classe social
0,18.14,5.94,0.00,4.74,15.38,3.94,0.63,6.34,4.48,6.29,...,13.77,17.68,17.83,6.52,0.00,18.07,3.72,1132.05,AC,Até 1908 (2)
1,22.45,5.17,0.00,3.59,25.21,5.36,1.21,12.16,6.47,7.29,...,18.73,19.62,21.96,8.08,0.00,11.82,4.07,1443.81,AC,Mais de 1908 a 2862
2,16.37,3.85,0.00,3.47,21.31,1.88,0.79,7.29,11.35,2.90,...,36.30,20.60,24.62,26.27,0.17,40.23,3.99,1578.39,AC,Mais de 2862 a 5724
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,38.79,3.52,0.00,0.70,21.68,1.63,3.74,12.14,4.17,4.06,...,3.30,3.31,0.00,40.45,0.00,4.92,3.48,1245.30,TO,Mais de 5724 a 9540
160,17.37,2.08,0.00,0.17,25.58,5.93,0.00,2.66,16.99,4.02,...,8.95,1.21,0.00,0.00,0.00,7.88,3.43,1239.49,TO,Mais de 9540 a 14310
161,51.45,20.18,0.00,0.00,38.20,1.61,0.00,0.00,36.59,5.70,...,2.43,1.72,0.00,0.00,0.00,0.00,3.01,1339.79,TO,Mais de 14310 a 23850


In [7]:
classes_sociais = ["Até 1908 (2)", "Mais de 1908 a 2862", "Mais de 2862 a 5724", "Mais de 5724 a 9540", "Mais de 9540 a 14310", "Mais de 14310 a 23850"]

## Análise de regressão linear

In [8]:
def regressao_linear_classes_sociais(df, classe_social):

    lista_colunas_afetam_valor_total = df.columns[:-3]

    X = df[df['Classe social'] == classe_social][lista_colunas_afetam_valor_total]
    Y = df[df['Classe social'] == classe_social]["Total gasto por classes"]

    lm = LinearRegression()
    lm.fit(X, Y)

    r2 = lm.score(X.values, Y)

    print("Coeficiente estimado:", lm.coef_)
    print("\nR2 (score):", r2)

    print("\nIntercept: %.5f" % lm.intercept_)

    equacao_str = ""
    for coef, feat in zip(lm.coef_,lista_colunas_afetam_valor_total):
        equacao_str += " + (%.5f * '%s')" % (coef, feat)

    print("\nEquação do 'valor-mensal' em termos de todos os atributos analisados:\n\n  f(x) = %.5f%s" % (lm.intercept_, equacao_str))

    print("\nCoeficiente de determinação é: %.5f, ou seja, %.5f%% do 'valor-mensal' não é explicada pelos atributos utilizados" % (r2, (1-r2)*100))

### Até 1908 (2)

In [9]:
regressao_linear_classes_sociais(df_estados_brasil, classes_sociais[0])

Coeficiente estimado: [ 1.01706171e+00  6.89806395e-01 -1.12197735e-03 -1.24655816e-03
  7.67268514e-03 -5.20695563e-02  1.77878393e-01  1.44785761e-01
  6.59923690e-02  5.89073263e-02  6.43813833e-02  3.11049068e-01
 -1.75648957e-03  1.61485318e-01  9.53992780e-01  3.52430270e-01
 -9.75842340e-03  2.55084402e-01  3.55235951e-01  2.02316390e-01
  2.26040714e-01  3.67536396e-01  2.55941839e-01  3.34376789e-02
 -9.89594443e-02 -5.20695563e-02  1.77878393e-01  1.44785761e-01
  6.59923690e-02  5.89073263e-02  6.43813833e-02  3.11049068e-01
 -1.75648957e-03  1.61485318e-01  1.16040646e+00  7.69362686e-01
  2.41400158e-01  6.53983013e-03  4.19550699e-03  1.32759301e-03
 -5.20695563e-02  1.77878393e-01  1.44785761e-01  6.59923690e-02
  5.89073263e-02  6.43813833e-02  3.11049068e-01 -1.75648957e-03
  1.61485318e-01  5.48090351e-01  3.52470927e-01  1.22464458e-01
  5.38909257e-03 -5.20695563e-02  1.77878393e-01  1.44785761e-01
  6.59923690e-02  5.89073263e-02  6.43813833e-02  3.11049068e-01
 -1

### Mais de 1908 a 2862

In [10]:
regressao_linear_classes_sociais(df_estados_brasil, classes_sociais[1])

Coeficiente estimado: [ 4.60236412e-01  7.53581878e-01 -7.89906987e-03  1.32502534e-02
  1.64097263e-02  3.73763465e-03  2.19723817e-01  3.07711748e-01
  5.97520508e-02  1.64254610e-02 -2.51886465e-02  2.89123563e-01
 -1.84335549e-03 -5.14270419e-02  6.59017364e-01  1.45010152e-01
  2.46507515e-01  1.08013478e-02  2.56109276e-01  1.36384449e-01
  3.44610119e-01  4.46493310e-01  1.61439208e-01  8.37434515e-02
 -1.99547974e-02  3.73763465e-03  2.19723817e-01  3.07711748e-01
  5.97520508e-02  1.64254610e-02 -2.51886465e-02  2.89123563e-01
 -1.84335549e-03 -5.14270419e-02  1.01391683e+00  6.71899572e-01
 -3.98635138e-02  7.04957417e-02  1.96284888e-01  1.13338071e-02
  3.73763465e-03  2.19723817e-01  3.07711748e-01  5.97520508e-02
  1.64254610e-02 -2.51886465e-02  2.89123563e-01 -1.84335549e-03
 -5.14270419e-02  4.72225399e-01  1.30052949e-01  2.31746384e-01
  5.18904170e-02  3.73763465e-03  2.19723817e-01  3.07711748e-01
  5.97520508e-02  1.64254610e-02 -2.51886465e-02  2.89123563e-01
 -1

### Mais de 2862 a 5724

In [11]:
regressao_linear_classes_sociais(df_estados_brasil, classes_sociais[2])

Coeficiente estimado: [ 1.02623559e+00  6.33059104e-01 -3.93715003e-03  4.73594877e-03
  4.66355289e-02  4.36269792e-02  3.96840290e-02  2.10569367e-01
  2.90605738e-01  3.12358147e-02  7.96483371e-02  1.87907037e-01
  5.66841273e-02  9.49787522e-02  7.32968080e-01  3.61404405e-01
  4.72161557e-02 -1.36473861e-02  3.38120831e-01 -7.79055356e-04
  1.78045144e-01  3.68677950e-01  1.46667883e-01  1.30676191e-01
  4.93440650e-02  4.36269792e-02  3.96840290e-02  2.10569367e-01
  2.90605738e-01  3.12358147e-02  7.96483371e-02  1.87907037e-01
  5.66841273e-02  9.49787522e-02  5.46219210e-01  2.22724084e-01
  1.02351051e-01  1.11218308e-02 -1.49149767e-01  4.85455305e-02
  4.36269792e-02  3.96840290e-02  2.10569367e-01  2.90605738e-01
  3.12358147e-02  7.96483371e-02  1.87907037e-01  5.66841273e-02
  9.49787522e-02  9.56234548e-01  4.84970139e-01  1.07372790e-01
  7.35666639e-02  4.36269792e-02  3.96840290e-02  2.10569367e-01
  2.90605738e-01  3.12358147e-02  7.96483371e-02  1.87907037e-01
  5

### Mais de 5724 a 9540

In [12]:
regressao_linear_classes_sociais(df_estados_brasil, classes_sociais[3])

Coeficiente estimado: [ 1.30707798  0.17350135  0.01022792 -0.00967183  0.01510121  0.11732638
  0.13133646  0.20923183  0.26751117  0.02041073  0.00533109  0.12810516
  0.00246186  0.21238575  1.08079191  0.40316188  0.11614976  0.16953885
  0.39245622 -0.10939782  0.0320789   0.43162434  0.27438451  0.09030411
 -0.06489762  0.11732638  0.13133646  0.20923183  0.26751117  0.02041073
  0.00533109  0.12810516  0.00246186  0.21238575  0.61488291  0.13632195
  0.29033517 -0.02055327 -0.08801914  0.05055993  0.11732638  0.13133646
  0.20923183  0.26751117  0.02041073  0.00533109  0.12810516  0.00246186
  0.21238575  0.79414186  0.37831835  0.20252667 -0.05495736  0.11732638
  0.13133646  0.20923183  0.26751117  0.02041073  0.00533109  0.12810516
  0.00246186  0.21238575  1.51702357  0.41915344  0.10847901  0.24734555
  0.7414889   1.8220033   0.52655655  0.3385654  -0.00418265  0.57003592
  0.37152815  0.11732638  0.13133646  0.20923183  0.26751117  0.02041073
  0.00533109  0.12810516  0.0

### Mais de 9540 a 14310

In [13]:
regressao_linear_classes_sociais(df_estados_brasil, classes_sociais[4])

Coeficiente estimado: [ 2.76556765e-01  1.50017299e-01 -4.11898967e-03  4.65435305e-02
  1.62374826e-03  2.27723546e-01  8.58035770e-02  1.72039589e-01
  1.90043105e-01  5.19156018e-02  2.32048575e-01  9.95519759e-02
  2.28186285e-01  2.24631938e-01  4.41383966e-01  8.10419074e-02
  1.02694389e-01  4.36949533e-02  2.14073201e-01  2.74370919e-01
  1.39437889e-01  3.66638560e-01  1.81985996e-01  8.62816403e-02
  1.27453243e-02  2.27723546e-01  8.58035770e-02  1.72039589e-01
  1.90043105e-01  5.19156018e-02  2.32048575e-01  9.95519759e-02
  2.28186285e-01  2.24631938e-01  1.43681740e-01  8.62534062e-02
 -2.79733011e-02 -8.67554116e-02  7.75458595e-02  6.54895983e-02
  2.27723546e-01  8.58035770e-02  1.72039589e-01  1.90043105e-01
  5.19156018e-02  2.32048575e-01  9.95519759e-02  2.28186285e-01
  2.24631938e-01  6.13454368e-01  1.85201924e-01  3.41851917e-02
  2.03655233e-01  2.27723546e-01  8.58035770e-02  1.72039589e-01
  1.90043105e-01  5.19156018e-02  2.32048575e-01  9.95519759e-02
  2

### Mais de 14310 a 23850

In [14]:
regressao_linear_classes_sociais(df_estados_brasil, classes_sociais[5])

Coeficiente estimado: [ 0.50431409  0.3637596   0.02295893 -0.15312258 -0.00550576  0.0932568
  0.2578368   0.12157352  0.12301437  0.04712972  0.11998703  0.03314128
  0.1947924   0.15606072  1.01166282  0.21555638  0.12464651  0.29290546
  0.37829097  0.05796769  0.07169487  1.06780944  0.5616547   0.28677714
 -0.03854126  0.0932568   0.2578368   0.12157352  0.12301437  0.04712972
  0.11998703  0.03314128  0.1947924   0.15606072  0.90631613  0.1094527
  0.38921969  0.28570218 -0.19565368  0.139719    0.0932568   0.2578368
  0.12157352  0.12301437  0.04712972  0.11998703  0.03314128  0.1947924
  0.15606072  1.36157687  0.99445063  0.18004054  0.06475234  0.0932568
  0.2578368   0.12157352  0.12301437  0.04712972  0.11998703  0.03314128
  0.1947924   0.15606072  1.30018579  0.55704884 -0.03235079  0.26300311
  0.51247354  1.59456339  0.66611238  0.52989996  0.21909969  0.30582586
 -0.17413391  0.0932568   0.2578368   0.12157352  0.12301437  0.04712972
  0.11998703  0.03314128  0.194792